In [116]:
import urllib
import requests
import os
from copy import deepcopy as mcopy
from music21 import *

In [5]:
# Initialize Music21 and 

defaults.author = ''
defaults.title = ''

def send_m21_object(obj, scoreName):
    xml = musicxml.m21ToString.fromMusic21Object(obj)
    headers = {'Content-Type': 'text/xml'}
    url = urllib.parse.urljoin('http://ks-images:3000/score/', scoreName)
    return requests.post(url, 
                        data=xml,
                        headers=headers,
                        timeout=5)

In [194]:
def append_octave_up(stream):
    """Append a copy of the stream an octave up (in place)"""
    for n in stream.notes:
        stream.append(n.transpose(12))
    return stream

def all_but_last(a_stream):
    """Return a new stream that includes all but the last note.
    Discard any non-note items in the stream.
    """
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    if len(notes) <= 1:
        return result
    result.append(notes[:-1])
    return result

def all_but_first_and_last(a_stream):
    """Return a new stream of the same type that includes all but
    the first and last rests/notes"""
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    if len(notes) <= 2:
        return result
    result.append(notes[1:-1])
    return result

def reverse_notes(a_stream):
    """Return a new stream with the notes and rests reversed."""
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    notes.reverse()
    result.append(notes)
    return result

def all_but_first(a_stream):
    """"""
    return reverse_notes(all_but_last(reverse_notes(a_stream)))

def arp_forward_rev_x_times(a_stream, x):
    """Arpeggiate a stream of notes x times. Playback the stream
    in reverse order, without playing the last note in the
    original stream twice. Finish on the note where we started.
    go through and back x times."""
    
    Class = a_stream.__class__
    result = Class()

    # All but last reversed
    ablr = reverse_notes(all_but_last(a_stream))
    result.append(a_stream)
    result.append(ablr)

    appendage = all_but_first(mcopy(result).flat)    
    result.repeatAppend(appendage, x - 1)
    return result.flat


In [199]:
s1 = stream.Part()
noteList = [note.Note(n, type='16th') for n in ['C3', 'D3', 'G3', 'A3']]
s1.append(noteList)
# append_octave_up(s1)
part1 = arp_forward_rev_x_times(s1, 4)
send_m21_object(part1, 'a')

<Response [200]>

In [178]:
s = stream.Stream()
s.append(note.Note('a4'))
s.append(note.Note('b4'))
s.append(note.Note('c5'))
s.show('text')
print('asd')
reverse_notes(s).show('text')
print('all but last (reverse)')
all_but_last(reverse_notes(s).sorted).sorted.show('text')

print('all but first')
all_but_first(s).show('text')


{0.0} <music21.note.Note A>
{1.0} <music21.note.Note B>
{2.0} <music21.note.Note C>
asd
{0.0} <music21.note.Note C>
{1.0} <music21.note.Note B>
{2.0} <music21.note.Note A>
all but last (reverse)
notes
[<music21.note.Note C>, <music21.note.Note B>, <music21.note.Note A>]
{0.0} <music21.note.Note C>
{1.0} <music21.note.Note B>
all but first
notes
[<music21.note.Note C>, <music21.note.Note B>, <music21.note.Note A>]
{0.0} <music21.note.Note B>
{1.0} <music21.note.Note C>
